# 📓 04 Machine Learning Geoespacial

Este notebook desarrolla modelos predictivos para estimar la densidad de servicios urbanos basándose en características geográficas y espaciales.

**Objetivo:** Predecir el número de amenidades (`count`) en cada celda utilizando variables como distancia al centro, densidad de edificios y densidad vial.

---

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, GroupKFold, train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# --- CONFIGURACIÓN DE ENTORNO ---
load_dotenv("../.env")
db_url = f"postgresql://{os.getenv('POSTGRES_USER')}:{os.getenv('POSTGRES_PASSWORD')}@{os.getenv('POSTGRES_HOST', 'localhost')}:5432/{os.getenv('POSTGRES_DB')}"
engine = create_engine(db_url)

print("Entorno de ML listo.")

## 1. Feature Engineering Espacial

Para que el modelo aprenda la estructura urbana, necesitamos generar variables explicativas (FEATURES) para cada celda.

In [ ]:
print("Cargando datos base desde PostGIS...")
grid = gpd.read_postgis("SELECT * FROM raw_data.amenity_clusters", engine, geom_col='geometry')
buildings = gpd.read_postgis("SELECT geometry FROM raw_data.osm_buildings", engine, geom_col='geometry')
edges = gpd.read_postgis("SELECT geometry FROM raw_data.osm_edges", engine, geom_col='geometry')
boundary = gpd.read_postgis("SELECT geometry FROM raw_data.comuna_boundaries", engine, geom_col='geometry')

# --- PROYECCIÓN A UTM ---
grid_utm = grid.to_crs(epsg=32719)
buildings_utm = buildings.to_crs(epsg=32719)
edges_utm = edges.to_crs(epsg=32719)
center_utm = boundary.to_crs(epsg=32719).centroid.iloc[0]

print("Datos proyectados y listos.")

### Generación de Variables

Crearemos las siguientes variables:
1. `dist_center`: Distancia de cada celda al centro de la comuna.
2. `building_count`: Cantidad de edificios dentro de la celda.
3. `road_count`: Conectividad vial.
4. `x`, `y`: Coordenadas geográficas.

In [ ]:
print("Calculando features espaciales...")
grid_utm['dist_center'] = grid_utm.geometry.centroid.distance(center_utm)
grid_utm['x'] = grid_utm.geometry.centroid.x
grid_utm['y'] = grid_utm.geometry.centroid.y

joined_b = gpd.sjoin(grid_utm, buildings_utm, how='left', predicate='intersects')
grid_utm['building_count'] = joined_b.groupby(joined_b.index).size() - 1

grid_utm['road_count'] = gpd.sjoin(grid_utm, edges_utm, how='left', predicate='intersects').groupby(level=0).size() - 1

print("Features generadas exitosamente.")

## 2. Entrenamiento del Modelo (Random Forest)

Entrenaremos un regresor para predecir el número de servicios.

In [ ]:
features = ['dist_center', 'x', 'y', 'building_count', 'road_count']
X = grid_utm[features].fillna(0)
y = grid_utm['count']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print(f"R² Score: {r2_score(y_test, y_pred):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")

## 3. Importancia de Variables y Guardado

Visualizamos que factores influyen más y guardamos el modelo.

In [ ]:
importances = pd.Series(rf.feature_importances_, index=features).sort_values(ascending=False)
importances.plot(kind='bar', title='Importancia de Variables')
plt.show()

grid_utm['prediction'] = rf.predict(X)
grid_wgs84 = grid_utm.to_crs(epsg=4326)
grid_wgs84.to_postgis("ml_predictions", engine, schema='raw_data', if_exists='replace')
print("Predicciones exportadas.")

## 4. Conclusiones del Modelado

1. **Poder Predictivo:** El modelo logra capturar la lógica de distribución de servicios basándose únicamente en atributos físicos del entorno.
2. **Variables Clave:** La densidad de edificaciones y la distancia al centro administrativo son los mejores predictores de la densidad de servicios.
3. **Aplicabilidad:** Las predicciones permiten identificar zonas donde, dado su desarrollo urbano, 'debería' haber más servicios de los que existen hoy.